In [ ]:
# ===============================
# Import Libraries
# ===============================
import pandas as pd
from textblob import TextBlob
from datetime import timedelta

In [13]:
# ===============================
# Load NASDAQ Data
# ===============================
nasdaq = pd.read_csv('1M_nasdaq_data.csv', parse_dates=['Date'], index_col='Date')
nasdaq.index = pd.to_datetime(nasdaq.index, utc=True)
nasdaq.index = nasdaq.index.tz_convert(None)
nasdaq.index = nasdaq.index.normalize()  # Keep only date part

nasdaq.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI,MACD,MACD_Signal,Bollinger_Width,MFI,EMA_20,ATR_14,ADX_14,OBV,Stoch_%K,Stoch_%D
Date,,,,,,,,,,,,,,,,,,
2022-02-18,13735.400391,13762.400391,13465.559570,13548.070312,4501940000,0.0,0.0,37.605558,-248.117301,-249.267109,1253.375229,49.738045,14090.924580,367.781455,45.477637,-18176870000,7.903328,27.924491
2022-02-22,13424.360352,13618.719727,13249.650391,13381.519531,4849170000,0.0,0.0,35.439178,-275.591262,-254.531940,1347.113386,42.333297,14023.362194,367.873447,45.657894,-23026040000,10.466559,8.173484
2022-02-23,13511.750000,13533.780273,13032.169922,13037.490234,4639330000,0.0,0.0,31.413556,-321.419737,-267.909499,1522.599029,35.004783,13929.469627,377.426083,46.116330,-27665370000,0.360116,6.243334
2022-02-24,12587.879883,13486.110352,12587.879883,13473.589844,6180200000,0.0,0.0,40.621333,-318.873761,-278.102352,1536.702848,34.192076,13886.052505,414.626396,47.045394,-21485170000,46.090406,18.972360
2022-02-25,13485.259766,13696.860352,13358.290039,13694.620117,4628720000,0.0,0.0,44.675353,-295.613117,-281.604505,1466.682210,34.549627,13867.820849,409.193818,47.113951,-16856450000,57.592337,34.680953


In [14]:
# ===============================
# Load News Content
# ===============================
news_df = pd.read_csv('2M_scraped_news.csv')

# change name of column published_date to date
news_df.rename(columns={'published_date': 'date'}, inplace=True)

news_df.index = pd.to_datetime(news_df['date'], utc=True)
news_df.index = news_df.index.tz_convert(None)
news_df.index = news_df.index.normalize()  # Keep only date part

news_df.head()

,title,description,url,date,source
date,,,,,
2023-01-31,Trump’s Abuse of Recount Funds Shows the Need ...,Trump’s Abuse of Recount Funds Shows the Need ...,https://news.google.com/rss/articles/CBMikgFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://campaignlegal.org', 'title':..."
2023-02-22,Trump criticizes federal response to Ohio trai...,Trump criticizes federal response to Ohio trai...,https://news.google.com/rss/articles/CBMiugFBV...,"Wed, 22 Feb 2023 08:00:00 GMT","{'href': 'https://apnews.com', 'title': 'AP Ne..."
2023-01-30,Fani Willis is likely going to indict Trump so...,Fani Willis is likely going to indict Trump so...,https://news.google.com/rss/articles/CBMi1gFBV...,"Mon, 30 Jan 2023 08:00:00 GMT","{'href': 'https://www.citizensforethics.org', ..."
2023-02-14,"Trump, after reported subpoena, defends keepin...","Trump, after reported subpoena, defends keepin...",https://news.google.com/rss/articles/CBMirgFBV...,"Tue, 14 Feb 2023 08:00:00 GMT","{'href': 'https://www.reuters.com', 'title': '..."
2023-01-31,Trump vows to 'stop' gender-affirming care for...,Trump vows to 'stop' gender-affirming care for...,https://news.google.com/rss/articles/CBMivwFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://www.nbcnews.com', 'title': '..."


In [15]:
# ===============================
# Move Saturday and Sunday news to Friday
# ===============================
for date in news_df.index:
    weekday = date.weekday()
    if weekday == 5:  # Saturday
        new_date = date - timedelta(days=1)
        news_df.loc[date, 'date'] = new_date
    elif weekday == 6:  # Sunday
        new_date = date - timedelta(days=2)
        news_df.loc[date, 'date'] = new_date

news_df.head()

,title,description,url,date,source
date,,,,,
2023-01-31,Trump’s Abuse of Recount Funds Shows the Need ...,Trump’s Abuse of Recount Funds Shows the Need ...,https://news.google.com/rss/articles/CBMikgFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://campaignlegal.org', 'title':..."
2023-02-22,Trump criticizes federal response to Ohio trai...,Trump criticizes federal response to Ohio trai...,https://news.google.com/rss/articles/CBMiugFBV...,"Wed, 22 Feb 2023 08:00:00 GMT","{'href': 'https://apnews.com', 'title': 'AP Ne..."
2023-01-30,Fani Willis is likely going to indict Trump so...,Fani Willis is likely going to indict Trump so...,https://news.google.com/rss/articles/CBMi1gFBV...,"Mon, 30 Jan 2023 08:00:00 GMT","{'href': 'https://www.citizensforethics.org', ..."
2023-02-14,"Trump, after reported subpoena, defends keepin...","Trump, after reported subpoena, defends keepin...",https://news.google.com/rss/articles/CBMirgFBV...,"Tue, 14 Feb 2023 08:00:00 GMT","{'href': 'https://www.reuters.com', 'title': '..."
2023-01-31,Trump vows to 'stop' gender-affirming care for...,Trump vows to 'stop' gender-affirming care for...,https://news.google.com/rss/articles/CBMivwFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://www.nbcnews.com', 'title': '..."


### TextBlob Sentiment

In [16]:
# ===============================
# Calculate Sentiment
# ===============================
def calculate_sentiment(text):
    if not isinstance(text, str) or text.strip() == '':
        return 0
    return TextBlob(text).sentiment.polarity

news_df['content_sentiment'] = news_df['description'].apply(calculate_sentiment)

news_df.head()

,title,description,url,date,source,content_sentiment
date,,,,,,
2023-01-31,Trump’s Abuse of Recount Funds Shows the Need ...,Trump’s Abuse of Recount Funds Shows the Need ...,https://news.google.com/rss/articles/CBMikgFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://campaignlegal.org', 'title':...",0.066667
2023-02-22,Trump criticizes federal response to Ohio trai...,Trump criticizes federal response to Ohio trai...,https://news.google.com/rss/articles/CBMiugFBV...,"Wed, 22 Feb 2023 08:00:00 GMT","{'href': 'https://apnews.com', 'title': 'AP Ne...",0.000000
2023-01-30,Fani Willis is likely going to indict Trump so...,Fani Willis is likely going to indict Trump so...,https://news.google.com/rss/articles/CBMi1gFBV...,"Mon, 30 Jan 2023 08:00:00 GMT","{'href': 'https://www.citizensforethics.org', ...",0.000000
2023-02-14,"Trump, after reported subpoena, defends keepin...","Trump, after reported subpoena, defends keepin...",https://news.google.com/rss/articles/CBMirgFBV...,"Tue, 14 Feb 2023 08:00:00 GMT","{'href': 'https://www.reuters.com', 'title': '...",0.000000
2023-01-31,Trump vows to 'stop' gender-affirming care for...,Trump vows to 'stop' gender-affirming care for...,https://news.google.com/rss/articles/CBMivwFBV...,"Tue, 31 Jan 2023 08:00:00 GMT","{'href': 'https://www.nbcnews.com', 'title': '...",0.000000


### NLTK Sentiment

In [ ]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()
# define a function to get the sentiment of a text
def get_sentiment(text):
    # get the sentiment polarity
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    # get the sentiment score
    sentiment_score = sia.polarity_scores(text)
    return sentiment, sentiment_score['compound']

# apply the function to the text column
news_df['content_sentiment'], news_df['sentiment_score'] = zip(*news_df['description'].apply(get_sentiment))

In [19]:
# ===============================
# Aggregate Daily Sentiment
# ===============================
daily_sentiment = news_df.groupby(news_df.index).agg(
    Daily_Sentiment=('content_sentiment', 'mean'),
    Article_Count=('content_sentiment', 'count')
)

# ===============================
# Print Summary
# ===============================
print("\n📊 Daily Sentiment Summary (After Moving Weekend to Friday):")
print(daily_sentiment)


📊 Daily Sentiment Summary (After Moving Weekend to Friday):
            Daily_Sentiment  Article_Count
date                                      
2023-01-26         0.000000              1
2023-01-27        -0.329167              5
2023-01-28         0.131566              6
2023-01-29         0.068182              2
2023-01-30         0.039394              9
...                     ...            ...
2025-03-26         0.012003              8
2025-03-27         0.087652             10
2025-03-28         0.007353             17
2025-03-29         0.011111              1
2025-03-30         0.045455              3

[522 rows x 2 columns]


In [20]:
# ===============================
# Build Sentiment Features for NASDAQ Dates
# ===============================
sentiment_features = pd.DataFrame(index=nasdaq.index)

for date in nasdaq.index:
    if date >= pd.Timestamp('2023-01-01'):

        weekday = date.weekday()  # Monday = 0, Sunday = 6

        if weekday <= 3:  # Monday to Thursday
            t1 = date - timedelta(days=1)
            t2 = date - timedelta(days=2)
            t3 = date - timedelta(days=3)
        else:  # Friday
            t1 = date
            t2 = date - timedelta(days=1)
            t3 = date - timedelta(days=2)

        # Lookup sentiments safely
        s_t1 = daily_sentiment.loc[daily_sentiment.index == t1, 'Daily_Sentiment'].values
        s_t2 = daily_sentiment.loc[daily_sentiment.index == t2, 'Daily_Sentiment'].values
        s_t3 = daily_sentiment.loc[daily_sentiment.index == t3, 'Daily_Sentiment'].values

        s_t1 = s_t1[0] if len(s_t1) > 0 else 0
        s_t2 = s_t2[0] if len(s_t2) > 0 else 0
        s_t3 = s_t3[0] if len(s_t3) > 0 else 0

        # Save features
        sentiment_features.loc[date, 'Sentiment_T1'] = s_t1
        sentiment_features.loc[date, 'Sentiment_T2'] = s_t2
        sentiment_features.loc[date, 'Sentiment_T3'] = s_t3
        sentiment_features.loc[date, 'Sentiment_3DayAVG'] = (s_t1 + s_t2 + s_t3) / 3

sentiment_features.tail()

,Sentiment_T1,Sentiment_T2,Sentiment_T3,Sentiment_3DayAVG
Date,,,,
2025-03-24,0.084848,0.000000,0.052381,0.045743
2025-03-25,0.000000,0.084848,0.000000,0.028283
2025-03-26,0.040000,0.000000,0.084848,0.041616
2025-03-27,0.012003,0.040000,0.000000,0.017334
2025-03-28,0.007353,0.087652,0.012003,0.035669


In [21]:
# ===============================
# Append Sentiment to NASDAQ
# ===============================
nasdaq = nasdaq.join(sentiment_features, rsuffix='_new')

# ===============================
# Save Updated NASDAQ
# ===============================
nasdaq.to_csv('3M_nasdaq_sentiment.csv')

print("\nSentiment features successfully appended to '3M_nasdaq_sentiment.csv'.")
print(nasdaq.tail(10))


Sentiment features successfully appended to '3M_nasdaq_sentiment.csv'.
                    Open          High           Low         Close  \
Date                                                                 
2025-03-17  17722.539062  17925.939453  17645.869141  17808.660156   
2025-03-18  17662.570312  17682.919922  17431.669922  17504.119141   
2025-03-19  17590.119141  17917.539062  17533.939453  17750.789062   
2025-03-20  17586.060547  17903.199219  17576.820312  17691.630859   
2025-03-21  17519.839844  17798.060547  17474.810547  17784.050781   
2025-03-24  18046.189453  18210.050781  18030.679688  18188.589844   
2025-03-25  18207.970703  18281.130859  18170.419922  18271.859375   
2025-03-26  18217.330078  18236.550781  17837.259766  17899.019531   
2025-03-27  17811.240234  17988.019531  17743.800781  17804.029297   
2025-03-28  17722.089844  17763.279297  17283.060547  17322.990234   

                Volume  Dividends  Stock Splits        RSI        MACD  \
Date         